In [14]:
import os
import pandas as pd
import fitsio
import time
import ray
import numpy as np
import threading
from pathlib import Path, PurePath

In [2]:
def extract_coincidentals(spikes_list, coords_id, idx):
    # Spikes coordinates at given wavelength index
    spikes_w = spikes_list[idx]
    # Associated neighbour coordinates
    nb_pixels = coords_id[spikes_w[0, :], :]
    # Sublist of spikes data that will excludes the one serving as template
    spikes_sublist = spikes_list[:idx] + spikes_list[idx + 1:]
    # Coincidental cross-referencing.
    mask_w_arr = np.array([np.isin(nb_pixels, coords_id[spikes[0, :], :]).any(axis=1) for spikes in spikes_sublist])
    select_pixels = mask_w_arr.any(axis=0)
    coords_w = spikes_w[0, select_pixels]
    w_tables = np.insert(mask_w_arr[:, select_pixels], idx, True, axis=0)
    # Retrieve intensity values for the selected coordinates
    intensities = spikes_w[1:, select_pixels]
    arr_w = np.concatenate([coords_w[np.newaxis, ...], intensities, w_tables], axis=0)
    arr_w = np.insert(arr_w, 3, idx, axis=0)

    return arr_w

In [33]:
def write_to_parquet(df_list, df_path):
    df = pd.concat(df_list)
    df.to_parquet(df_path, engine='pyarrow', compression=None)
    return None

In [4]:
@ray.remote
def process_group(coords, fpaths, group_n):
    spikes_list = [fitsio.read(os.path.join(os.environ['SPIKESDATA'], f)) for f in fpaths]
    group_data = np.concatenate([extract_coincidentals(spikes_list, coords, i) for i in range(7)], axis=1)
    column_names = ['coords', 'int1', 'int2', 'wref', 'w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6']
    coincidental_spikes_df = pd.DataFrame(group_data.T, columns=column_names)
    coincidental_spikes_df['GroupNumber'] = group_n
    return coincidental_spikes_df

In [5]:
nx, ny = 4096, 4096
# List of relative 2D coordinates for 8-neighbour connectiviy (9-element list). 1st one is the origin pixel.
coords_8nb = np.array([[0, 0], [-1, 0], [-1, -1], [0, -1], [1, -1], [1, 0], [1, 1], [0, 1], [-1, 1]])
# Array of 2D coordinates for a 4096 x 4096 array. Matrix convention is kept. [rows, cols] = [y-axis, x-axis]
coords_1d = np.arange(nx * ny)
coordy, coordx = np.unravel_index(coords_1d, [ny, nx]) # also possible by raveling a meshgrid() output
coords2d = np.array([coordy, coordx])
# Create the array of 2D coordinates of 8-neighbours associated with each pixel.
# pixel 0 has 8 neighbour + itself, pixel 1 has 8 neighbour + itself, etc...
coords2d_8nb = coords2d[np.newaxis, ...] + coords_8nb[..., np.newaxis]
# Handle off-edges coordinates by clipping to the edges, operation done in-place. Here, square detector assumed.
# to per-axis clipping if that ever changes for another instrument.
np.clip(coords2d_8nb, 0, nx-1, out=coords2d_8nb)
# Convert to 1D coordinates.
lookup_coords = np.array([coords2d_8nb[i, 0, :] * nx + coords2d_8nb[i, 1, :] for i in range(len(coords_8nb))],
                     dtype='int32', order='C').T

In [6]:
spikes_df = pd.read_parquet(os.path.join(os.environ['SPIKESDATA'], 'spikes_df_2010_filtered.parquet'), engine='pyarrow')
spikes_df2 = spikes_df.set_index(['GroupNumber', 'Time'])

In [7]:
ncpus = 60
ray.init(num_cpus=ncpus)

2020-02-13 10:06:57,947	WARNING worker.py:682 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2020-02-13 10:06:57,951	WARNING services.py:592 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-02-13 10:06:57,952	INFO resource_spec.py:212 -- Starting Ray with 333.98 GiB memory available for workers and up to 18.63 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-02-13 10:06:58,632	WARNING services.py:1080 -- Failed to start the dashboard. The dashboard requires Python 3 as well as 'pip install aiohttp psutil setproctitle grpcio'.


{'node_ip_address': '10.1.222.15',
 'redis_address': '10.1.222.15:55056',
 'object_store_address': '/tmp/ray/session_2020-02-13_10-06-57_950843_45442/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-02-13_10-06-57_950843_45442/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-02-13_10-06-57_950843_45442'}

In [37]:
outputdir = '/media/user/SPIKESDF/raphael/data/AIA_Spikes/parquet_dataframes/'

start, end = pd.Timestamp('2010-05-13 00:00:00', tz='UTC'), pd.Timestamp('2010-05-13 02:00:00', tz='UTC')
tinterval = pd.Interval(left=start, right=end)
groups = spikes_df['GroupNumber'].loc[(spikes_df['Time'] >= tinterval.left) & (spikes_df['Time'] < tinterval.right)].unique()
date = tinterval.left
day_dir = PurePath(outputdir, '{:d}/{:02d}'.format(date.year, date.month)).as_posix()
Path(day_dir).mkdir(parents=True, exist_ok=True)
df_path = PurePath(day_dir, 'df_coincidentals_{:d}_{:02d}_{:02d}.parquet'.format(date.year, date.month,
                                                                                 date.day)).as_posix()
print(len(groups))

600


In [38]:
fpaths_ = [spikes_df2['Path'].loc[group_n] for group_n in groups]

In [39]:
fpaths_id = ray.put(fpaths_)
coords_id = ray.put(lookup_coords)

In [40]:
%%time 
df_list = ray.get([process_group.remote(coords_id, fpaths, g) for g, fpaths in enumerate(fpaths_)])
write_to_parquet(df_list, df_path)

CPU times: user 5.36 s, sys: 1.37 s, total: 6.74 s
Wall time: 28.8 s


In [42]:
%time write_to_parquet(df_list, df_path)

CPU times: user 2.68 s, sys: 880 ms, total: 3.56 s
Wall time: 3.74 s


In [41]:
%%time 
df_list = ray.get([process_group.remote(coords_id, fpaths, g) for g, fpaths in enumerate(fpaths_)])
write_thread = threading.Thread(target=write_to_parquet, args=(df_list, df_path))
write_thread.start()
print('done')

done
CPU times: user 2.82 s, sys: 603 ms, total: 3.43 s
Wall time: 24.9 s


In [11]:
outputdir = os.environ['SPIKESDF']

KeyError: 'SPIKESDF'

In [43]:
ray.terminate()

AttributeError: module 'ray' has no attribute 'terminate'

In [44]:
ray.shutdown()